In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision as tv
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_auc_score

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import time
import os
import copy
import requests
import io
import csv
import random
from timm.data import create_dataset, create_loader
from timm.scheduler import StepLRScheduler


plt.ion()   # interactive mode

import timm 
from tqdm import tqdm

In [2]:
# Check if CUDA (GPU support) is available
is_cuda_available = torch.cuda.is_available()
print("Is CUDA (GPU) available:", is_cuda_available)

# If CUDA is available, print the GPU name(s)
if is_cuda_available:
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPU(s) available: {gpu_count}")
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU.")

Is CUDA (GPU) available: True
Number of GPU(s) available: 1
GPU 0: Quadro RTX 6000


In [4]:
# config 
input_size = 3, 224, 224
img_size = 224
num_classes = 2
batch_size = 128
interpolation = 'bicubic'

test_dir = '../Dataset/images/test_binary'

class_map = {
        'Normal': 0,
        'Cases': 1
        }

# create the train and etest datasets
test_dataset = create_dataset(name='', root=test_dir, split='validation', is_training=False, batch_size=batch_size, class_map = class_map)

test_len = len(test_dataset)
print('testidation set size: ' + str(test_len))

# resize images to fit the input of pretrained model
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((224*3, 224*3)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_dataset.transform = transform

# create data loaders 
loader_test = create_loader(
        test_dataset,
        input_size=input_size,
        batch_size=batch_size,
        is_training=False,
        interpolation=interpolation,
        num_workers=8,
        pin_memory=True)

# check if labels are loaded as defined
test_dataset.reader.class_to_idx

testidation set size: 25596


{'Normal': 0, 'Cases': 1}

In [5]:
# define model
model_name = 'resnet50.a1_in1k'

model = timm.create_model(model_name, pretrained=False, num_classes=num_classes) #, img_size=img_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # if print 'cuda' then GPU is used
model.to(device)

cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

## Load model

In [6]:
ckpt_save_path = 'model_result/model_checkpoint/ResNet50/MODEL_CKPT_29_resnet50.a1_in1k_20240407_153814.pt'
checkpoint = torch.load(ckpt_save_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
model.eval()
torch.cuda.empty_cache()

## Predict

In [8]:
test_labels = np.empty((0,))
test_predictions = np.empty((0,))

with torch.no_grad():
    for images, labels in tqdm(loader_test):
        images = images.to(device)
        labels = labels.cpu().numpy()
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.cpu().numpy()

        test_labels = np.concatenate((test_labels, labels))
        test_predictions = np.concatenate((test_predictions, predictions))

100%|██████████| 200/200 [03:51<00:00,  1.16s/it]


In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix

In [10]:
# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)

# AUC
auc = roc_auc_score(test_labels, test_predictions)
print("AUC:", auc)

# Recall
recall = recall_score(test_labels, test_predictions)
print("Recall:", recall)

# Precision
precision = precision_score(test_labels, test_predictions)
print("Precision:", precision)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, test_predictions).ravel()
print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)

# Specificity
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Accuracy: 0.6840521956555712
AUC: 0.6579414877280375
Recall: 0.7717190975532253
Precision: 0.7298353167448011
True Negative: 5366
False Positive: 4495
False Negative: 3592
True Positive: 12143
Sensitivity: 0.7717190975532253
Specificity: 0.5441638779028496
